# 0.**Librerias**

In [1]:
!pip install -U torch --index-url https://download.pytorch.org/whl/cu121
!pip install -U transformers accelerate datasets bitsandbytes triton peft trl \
    bert-score textstat numpy pandas
# ===============================
# ✅ LIMPIEZA DE SALIDA Y VERIFICACIÓN DE VERSIONES
# ===============================
import IPython
IPython.display.clear_output(wait=True)

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 128.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 143.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 101.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation:

In [1]:
import torch, transformers, peft, trl, accelerate, bitsandbytes, datasets

print("✅ Librerías cargadas correctamente:\n")
print(f"PyTorch:        {torch.__version__}")
print(f"Transformers:   {transformers.__version__}")
print(f"PEFT:           {peft.__version__}")
print(f"TRL:            {trl.__version__}")
print(f"Accelerate:     {accelerate.__version__}")
print(f"Datasets:       {datasets.__version__}")
print(f"BitsAndBytes:   {bitsandbytes.__version__}")

print("\n✨ Entorno listo para QLoRA 4-bit + SFTTrainer + TRL 0.24.0 ✨")

✅ Librerías cargadas correctamente:

PyTorch:        2.8.0+cu126
Transformers:   4.57.1
PEFT:           0.17.1
TRL:            0.24.0
Accelerate:     1.11.0
Datasets:       4.4.1
BitsAndBytes:   0.48.2

✨ Entorno listo para QLoRA 4-bit + SFTTrainer + TRL 0.24.0 ✨


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model
import glob, pandas as pd
import bitsandbytes as bnb
import os
import re
import random
import textwrap
from datasets import Dataset


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

# 1. Crear carpeta y entrar
!mkdir -p cochrane_data
%cd cochrane_data

# 2. Inicializar git vacío
!git init

# 3. Agregar remote
!git remote add -f origin https://github.com/feliperussi/bridging-the-gap-in-health-literacy.git

# 4. Activar modo sparse checkout
!git config core.sparseCheckout true

# 5. Indicar SOLO las carpetas y el archivo que queremos
!echo "data_collection_and_processing/Data Sources/Cochrane/train/pls" >> .git/info/sparse-checkout
!echo "data_collection_and_processing/Data Sources/Cochrane/train/non_pls" >> .git/info/sparse-checkout


# 6. Descargar solo esas partes del repo
!git pull origin main


/content/cochrane_data
hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/cochrane_data/.git/
Updating origin
remote: Enumerating objects: 72074, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 72074 (delta 0), reused 2 (delta 0), pack-reused 72071 (from 2)
Receiving objects: 100% (72074/72074), 315.90 MiB | 25.17 MiB/s, done.
Resolving deltas: 100% (2991/2991), done.
From https://github.com/feliperussi/bridging-the-gap-in-health-literacy
 * [new branch]

Gestionar Hugging Face

In [5]:
os.environ["HF_TOKEN"] = "hf"

In [6]:
from huggingface_hub import login
login(token=os.environ["HF_TOKEN"])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


# 1.Construir el dataset non_pls → pls desde las carpetas de GitHub

In [7]:
os.chdir("..")

Tomando datos de todo el repertorio de Cochrane

In [8]:
# Número máximo de pares
pares = 11

# === 1️⃣ Listar archivos ===
pls_files = sorted(glob.glob("cochrane_data/data_collection_and_processing/Data Sources/Cochrane/train/pls/*.txt"))
non_pls_files = sorted(glob.glob("cochrane_data/data_collection_and_processing/Data Sources/Cochrane/train/non_pls/*.txt"))

# === 2️⃣ Filtrar non-PLS que contengan 'accumulated' ===
non_pls_files = [f for f in non_pls_files if "accumulated" not in os.path.basename(f).lower()]

print(f"📚 Total PLS: {len(pls_files)}")
print(f"📂 Total non-PLS (sin 'accumulated'): {len(non_pls_files)}")

# === 3️⃣ Función para extraer el ID base (hasta antes de 'pub') ===
def extract_id(filename):
    """
    Extrae el identificador base del archivo hasta antes de 'pub'.
    Ejemplo:
      10.1002-14651858.CD000371.pub7-abstract.txt → 10.1002-14651858.CD000371
    """
    base = os.path.basename(filename)
    match = re.match(r"(.+?)\.pub\d+", base)
    return match.group(1) if match else base.split(".txt")[0]

# === 4️⃣ Crear diccionarios {id_base: ruta} ===
pls_dict = {extract_id(p): p for p in pls_files}
non_pls_dict = {extract_id(n): n for n in non_pls_files}

# === 5️⃣ Emparejar por ID base ===
data = []
for base_id, non_path in non_pls_dict.items():
    if base_id in pls_dict:
        pls_path = pls_dict[base_id]
        with open(pls_path, "r", encoding="utf-8") as f1, open(non_path, "r", encoding="utf-8") as f2:
            data.append({
                "id": base_id,
                "file_pls": os.path.basename(pls_path),
                "file_non_pls": os.path.basename(non_path),
                "pls": f1.read().strip(),
                "non_pls": f2.read().strip()
            })
    if len(data) >= pares:
        break

# === 6️⃣ Crear DataFrame ===
df = pd.DataFrame(data)

# === 7️⃣ Guardar resultados ===
output_path = "cochrane_pairs_clean.csv"
df.to_csv(output_path, index=False, encoding="utf-8")

print(f"\n✅ Emparejados {len(df)} pares por nombre base (sin 'accumulated')")
print(f"💾 Guardado en: {output_path}")
df.head(3)




📚 Total PLS: 16241
📂 Total non-PLS (sin 'accumulated'): 7251

✅ Emparejados 11 pares por nombre base (sin 'accumulated')
💾 Guardado en: cochrane_pairs_clean.csv


,id,file_pls,file_non_pls,pls,non_pls
0,10.1002-14651858.CD000006,10.1002-14651858.CD000006.pub2-pls.txt,10.1002-14651858.CD000006.pub2-abstract.txt,Absorbable stitches for repair of episiotomy a...,Background\nApproximately 70% of women will ex...
1,10.1002-14651858.CD000009,10.1002-14651858.CD000009.pub4-pls.txt_accumul...,10.1002-14651858.CD000009.pub4-abstract.txt_se...,Do acupuncture and related therapies help smok...,We included 38 studies. Based on three studies...
2,10.1002-14651858.CD000012,10.1002-14651858.CD000012.pub4-pls.txt,10.1002-14651858.CD000012.pub4-abstract.txt_se...,Alternative versus conventional institutional ...,"Ten trials involving 11,795 women met the incl..."


# 2.Test,train split y generacion del prompt

In [11]:
df = df.rename(columns={"non_pls": "source", "pls": "target"})

INSTR_PROMPT = """Using the following abstract of a biomedical study as input, generate a Plain Language Summary
(PLS) understandable by any patient, regardless of their health literacy. Ensure that the generated text
adheres to the following instructions which should be followed step-by-step:
a. Specific Structure: The generated PLS should be presented in a logical order, using the following
order:
1. Plain Title
2. Rationale
3. Trial Design
4. Results
b. Sections should be authored following these parameters:
1. Plain Title: Simplified title understandable to a layperson that summarizes the research that was
done.
2. Rationale: Include: background or study rationale providing a general description of the
condition, what it may cause or why it is a burden for the patients; the reason and main hypothesis
for the study; and why the study is needed, and why the study medication has the potential to
treat the condition.
3. Trial Design: Answer ‘How is this study designed?’ Include the description of the design,
description of study and patient population (age, health condition, gender), and the expected
amount of time a person will be in the study.
4. Results: Answer ‘What were the main results of the study’, include the benefits for the patients,
how the study was relevant for the area of study, and the conclusions from the investigator.
c. Consistency and Replicability: The generated PLS should be consistent regardless of the order of
sentences or the specific phrasing used in the input protocol text.
d. Compliance with Plain Language Guidelines: The generated PLS must follow all these plain
language guidelines:
• Have readability grade level of 6 or below.
• Do not have jargon. All technical or medical words or terms should be defined or broken down
into simple and logical explanations.
• Active voice, not passive.
• Mostly one or two syllable words.
• Sentences of 15 words or less.
• Short paragraphs of 3-5 sentences.
• Simple numbers (e.g., ratios, no percentages).
e. Do not invent Content: The AI model should not invent information. If the AI model includes data
other than the one given in the input abstract, the AI model should guarantee such data is verified and
real.
f. Aim for an approximate PLS length of 500-900 words.

Input abstract:

{source}

Output PLS:
"""

def format_example(row):
    prompt = INSTR_PROMPT.format(source=row["source"])
    return {"prompt": prompt, "response": row["target"]}

# Aplicar la función a cada fila del DataFrame
dataset = df.apply(format_example, axis=1).to_list()

# Crear Dataset de Hugging Face y dividir en train/test


hf_ds = Dataset.from_list(dataset).train_test_split(test_size=0.9, seed=42)

print(hf_ds)


DatasetDict({
    train: Dataset({
        features: ['prompt', 'response'],
        num_rows: 1
    })
    test: Dataset({
        features: ['prompt', 'response'],
        num_rows: 10
    })
})


## Mostrar uno al azar

In [15]:

# Mostrar 10 ejemplos aleatorios del conjunto de entrenamiento
num_ejemplos = 10

print(f"📊 Mostrando {num_ejemplos} ejemplos de 'prompt' y 'response' del dataset de entrenamiento:\n")
print("=" * 150)

# Seleccionamos 20 índices aleatorios sin repetición
indices = random.sample(range(len(hf_ds["test"])), num_ejemplos)

for i, idx in enumerate(indices, start=1):
    ejemplo = hf_ds["test"][idx]

    print(f"\n🔹 Ejemplo #{i} (índice {idx})")
    print("-" * 150)

    # --- PROMPT ---
    print("\n🟢 Prompt:")
    print(textwrap.fill(str(ejemplo.get("prompt", "")).replace("\\n", "\n"), width=150, replace_whitespace=False))

    # --- RESPONSE ---
    print("\n🔵 Response:")
    print(textwrap.fill(str(ejemplo.get("response", "")).replace("\\n", "\n"), width=150, replace_whitespace=False))

    print("=" * 150)


📊 Mostrando 10 ejemplos de 'prompt' y 'response' del dataset de entrenamiento:


🔹 Ejemplo #1 (índice 1)
------------------------------------------------------------------------------------------------------------------------------------------------------

🟢 Prompt:
Using the following abstract of a biomedical study as input, generate a Plain Language Summary
(PLS) understandable by any patient, regardless of
their health literacy. Ensure that the generated text
adheres to the following instructions which should be followed step-by-step:
a. Specific
Structure: The generated PLS should be presented in a logical order, using the following
order:
1. Plain Title
2. Rationale
3. Trial Design
4. Results
b. Sections should be authored following these parameters:
1. Plain Title: Simplified title understandable to a layperson that summarizes the research
that was
done.
2. Rationale: Include: background or study rationale providing a general description of the
condition, what it may cause or why

# 3.Modelo (Modelo Base sin training)

version Llama

In [16]:
MODELO="meta-llama/Llama-3.2-3B"

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForSeq2Seq, BitsAndBytesConfig, AutoModel,TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig
import torch.nn as nn
import torch.nn.functional as F
import gc
import os
import sys

print("\n" + "="*80)
print("🚀 CARGANDO EL MODELO")
print("="*80)

# 1. Cargar modelo base
model_id = MODELO
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

print(f"\n📥 Cargando {model_id}...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token


🚀 SETUP - OPCIÓN C FIXED

📥 Cargando meta-llama/Llama-3.2-3B...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

In [18]:
print(f"Dataset original size:")
print(f"   Train: {len(hf_ds['train'])}")
print(f"   Test: {len(hf_ds['test'])}")

Dataset original size:
   Train: 1
   Test: 10


Ejemplo de Generación

In [19]:
from transformers import GenerationConfig

prompt = """Using the following abstract of a biomedical study as input, generate a Plain Language Summary
(PLS) understandable by any patient, regardless of their health literacy. Ensure that the generated text
adheres to the following instructions which should be followed step-by-step:
a. Specific Structure: The generated PLS should be presented in a logical order, using the following
order:
1. Plain Title
2. Rationale
3. Trial Design
4. Results
b. Sections should be authored following these parameters:
1. Plain Title: Simplified title understandable to a layperson that summarizes the research that was
done.
2. Rationale: Include: background or study rationale providing a general description of the
condition, what it may cause or why it is a burden for the patients; the reason and main hypothesis
for the study; and why the study is needed, and why the study medication has the potential to
treat the condition.
3. Trial Design: Answer ‘How is this study designed?’ Include the description of the design,
description of study and patient population (age, health condition, gender), and the expected
amount of time a person will be in the study.
4. Results: Answer ‘What were the main results of the study’, include the benefits for the patients,
how the study was relevant for the area of study, and the conclusions from the investigator.
c. Consistency and Replicability: The generated PLS should be consistent regardless of the order of
sentences or the specific phrasing used in the input protocol text.
d. Compliance with Plain Language Guidelines: The generated PLS must follow all these plain
language guidelines:
• Have readability grade level of 6 or below.
• Do not have jargon. All technical or medical words or terms should be defined or broken down
into simple and logical explanations.
• Active voice, not passive.
• Mostly one or two syllable words.
• Sentences of 15 words or less.
• Short paragraphs of 3-5 sentences.
• Simple numbers (e.g., ratios, no percentages).
e. Do not invent Content: The AI model should not invent information. If the AI model includes data
other than the one given in the input abstract, the AI model should guarantee such data is verified and
real.
f. Aim for an approximate PLS length of 500-900 words.

Abstract Input:
The feline consumed the rodent.

Output PLS"""

# Tokenización y envío a GPU
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")

# Configuración de generación estable
gen_config = GenerationConfig(
    max_new_tokens=80,
    temperature=0.3,            # salida más determinista
    top_p=0.9,
    do_sample=False,            # greedy decoding (sin sampling)
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    repetition_penalty=1.8,     # penaliza repeticiones
    no_repeat_ngram_size=4,     # evita repeticiones de frases
)

# Generar texto
model.eval()
with torch.no_grad():
    outputs = model.generate(**inputs, generation_config=gen_config)

# Decodificar texto
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# 🔧 Limpiar el resultado
if "Output PLS:" in generated_text:
    simplified = generated_text.split("Output PLS:")[1]
    simplified = simplified.split("###")[0].strip()
else:
    simplified = generated_text.strip()

# Cortar si se alarga tras un punto
if "." in simplified:
    simplified = simplified.split(".")[0] + "."

print("🧩 Prompt:", prompt)
print("=" * 60)
print("💬 Simplificación limpia:")
print(simplified)
print("=" * 60)



The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🧩 Prompt: Using the following abstract of a biomedical study as input, generate a Plain Language Summary
(PLS) understandable by any patient, regardless of their health literacy. Ensure that the generated text
adheres to the following instructions which should be followed step-by-step:
a. Specific Structure: The generated PLS should be presented in a logical order, using the following
order:
1. Plain Title
2. Rationale
3. Trial Design
4. Results
b. Sections should be authored following these parameters:
1. Plain Title: Simplified title understandable to a layperson that summarizes the research that was
done.
2. Rationale: Include: background or study rationale providing a general description of the
condition, what it may cause or why it is a burden for the patients; the reason and main hypothesis
for the study; and why the study is needed, and why the study medication has the potential to
treat the condition.
3. Trial Design: Answer ‘How is this study designed?’ Include the description

# 4.Generación de Resumenes


## Preparar Dataset para pasar a DataFrame df_test para la Generación de Texto

In [20]:
print(hf_ds["test"].column_names)
df_test = hf_ds["test"].to_pandas()
print(df_test.columns)

import re
import pandas as pd

def extract_source_from_prompt(prompt):
    """
    Extrae el texto entre 'original text' y 'simplified text' del prompt.
    Robusto ante espacios, mayúsculas y saltos de línea.
    """
    match = re.search(
        r"Input abstract:\s*[:\-–]?\s*(.*?)\s*Output PLS:",
        prompt,
        re.IGNORECASE | re.DOTALL
    )
    if match:
        return match.group(1).strip()
    return ""

# Crear columna 'source' desde 'prompt'
df_test["source"] = df_test["prompt"].apply(extract_source_from_prompt)

# Mantener solo columnas necesarias
df_test = df_test[["source", "response"]]

# Mostrar ejemplo de verificación
print(df_test.head(2))

['prompt', 'response']
Index(['prompt', 'response'], dtype='object')
                                              source  \
0  Background\nIt is unclear whether blood pressu...   
1  Background\nIn acute asthma inhaled beta₂‐agon...   

                                            response  
0  Drug interventions for deliberately altering b...  
1  Holding chambers (spacers) versus nebulisers f...  


## Generación de texto para el set TEST

In [21]:
if "trainer" in locals() and hasattr(trainer, "model"):
    fine_tuned_model = trainer.model.eval()
else:
    fine_tuned_model = model.eval()
    print("Modelo base")

Modelo base


In [22]:
df_test["source"]

,source
0,Background\nIt is unclear whether blood pressu...
1,Background\nIn acute asthma inhaled beta₂‐agon...
2,Background\nApproximately 70% of women will ex...
3,Background\nExtracranial carotid artery stenos...
4,Background\nPatients treated with mechanical v...
5,Background\nWhilst the pharmacological profile...
6,"Ten trials involving 11,795 women met the incl..."
7,Background\nStroke is the third leading cause ...
8,Background\nStriae gravidarum (stretch marks d...
9,We included 38 studies. Based on three studies...


In [23]:
def generate_simplifications_v2(
    df,
    model,
    tokenizer,
    instr_prompt,
    batch_size=2,
    max_source_tokens=512,
    sim_threshold=0.75,
    verbose=True,
):
    """
    Genera textos simplificados con regeneración multinivel.
    Si el texto generado está vacío, intenta regenerar automáticamente.
    """

    import gc, torch, re
    from tqdm import tqdm
    from difflib import SequenceMatcher

    tokenizer.padding_side = "right"
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.clean_up_tokenization_spaces = True

    model.eval()
    model.config.use_cache = True
    model.config.output_hidden_states = False
    model.config.return_dict = True

    torch.cuda.empty_cache()
    gc.collect()

    if verbose:
        print("\n🚀 Generación iniciada (modo seguro, con detección de vacíos)...")

    # === FUNCIONES AUXILIARES ===
    def truncate_text(text, max_tokens=max_source_tokens):
        tokens = text.split()
        if len(tokens) <= max_tokens:
            return text
        head = tokens[:max_tokens // 2]
        tail = tokens[-max_tokens // 2:]
        return " ".join(head + ["[...]"] + tail)

    def safe_generate(prompts, gen_params):
        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1536
        ).to(model.device)

        with torch.inference_mode():
            outputs = model.generate(**inputs, **gen_params)

        texts = tokenizer.batch_decode(
            outputs,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
            errors="replace"
        )
        return texts

    def extract_simplified_text(text):
        match = re.search(r"Output PLS\s*[:\-–]?\s*(.*)", text, re.IGNORECASE | re.DOTALL)
        if match:
            simplified = match.group(1).strip()
        else:
            simplified = text.strip()
        simplified = re.sub(r"<\|endoftext\|>", "", simplified)
        simplified = re.sub(r"\s+", " ", simplified).strip()
        return simplified

    # === PARÁMETROS DE GENERACIÓN ===
    GEN_PARAMS_L1 = {"max_new_tokens": 256, "temperature": 0.7, "top_p": 0.9, "do_sample": False,
                     "num_beams": 1, "repetition_penalty": 1.1,
                     "eos_token_id": tokenizer.eos_token_id, "pad_token_id": tokenizer.pad_token_id}

    GEN_PARAMS_L2 = {"max_new_tokens": 256, "temperature": 1.1, "top_p": 0.85, "do_sample": True,
                     "num_beams": 1, "repetition_penalty": 1.0,
                     "eos_token_id": tokenizer.eos_token_id, "pad_token_id": tokenizer.pad_token_id}

    GEN_PARAMS_L3 = {"max_new_tokens": 256, "temperature": 0.7, "top_p": 0.9, "do_sample": False,
                     "num_beams": 4, "repetition_penalty": 1.1,
                     "eos_token_id": tokenizer.eos_token_id, "pad_token_id": tokenizer.pad_token_id}

    # === PREPARACIÓN ===
    df = df.copy()
    df["source_for_generation"] = df["source"].apply(lambda x: truncate_text(x))
    if verbose:
        print("✅ Columna 'source_for_generation' creada correctamente")

    # === GENERACIÓN PRINCIPAL ===
    generated_texts, is_copy_list, was_regenerated_list = [], [], []
    regeneration_level_list, similarity_scores = [], []
    regeneration_count = 0

    for i in tqdm(range(0, len(df), batch_size), desc="Generando simplificaciones"):
        if i % 10 == 0:
            torch.cuda.empty_cache()
            gc.collect()

        batch_sources = df["source"].iloc[i:i+batch_size].tolist()
        batch_sources_trunc = df["source_for_generation"].iloc[i:i+batch_size].tolist()
        batch_prompts = [instr_prompt.format(source=s) for s in batch_sources_trunc]

        full_texts = safe_generate(batch_prompts, GEN_PARAMS_L1)

        for j, full_text in enumerate(full_texts):
            simplified = extract_simplified_text(full_text)
            source_idx = i + j
            if source_idx >= len(df):
                continue

            # 🔍 --- NUEVO BLOQUE: REGENERACIÓN SI EL TEXTO ESTÁ VACÍO ---
            if not simplified or len(simplified.strip()) == 0:
                if verbose:
                    print(f"⚠️ Texto vacío detectado en índice {source_idx}. Regenerando...")
                single_prompt = instr_prompt.format(source=batch_sources_trunc[j])
                regen_text = safe_generate([single_prompt], GEN_PARAMS_L2)[0]
                simplified = extract_simplified_text(regen_text)
                # Si sigue vacío, prueba con L3
                if not simplified or len(simplified.strip()) == 0:
                    beam_text = safe_generate([single_prompt], GEN_PARAMS_L3)[0]
                    simplified = extract_simplified_text(beam_text)

            # === Calcular similitud y regeneraciones ===
            sim = SequenceMatcher(None, batch_sources[j].lower(), simplified.lower()).ratio()
            is_copy = sim > sim_threshold
            regen_level = 0

            if is_copy:
                regeneration_count += 1
                single_prompt = instr_prompt.format(source=batch_sources_trunc[j])

                regen_text = safe_generate([single_prompt], GEN_PARAMS_L2)[0]
                simplified_regen = extract_simplified_text(regen_text)
                sim_regen = SequenceMatcher(None, batch_sources[j].lower(), simplified_regen.lower()).ratio()

                if sim_regen <= sim_threshold:
                    simplified, sim, is_copy, regen_level = simplified_regen, sim_regen, False, 2
                else:
                    beam_text = safe_generate([single_prompt], GEN_PARAMS_L3)[0]
                    simplified_beam = extract_simplified_text(beam_text)
                    sim_beam = SequenceMatcher(None, batch_sources[j].lower(), simplified_beam.lower()).ratio()

                    if sim_beam <= sim_threshold:
                        simplified, sim, is_copy, regen_level = simplified_beam, sim_beam, False, 3
                    else:
                        if sim_beam < sim_regen:
                            simplified, sim, regen_level = simplified_beam, sim_beam, 3
                        else:
                            simplified, sim, regen_level = simplified_regen, sim_regen, 2

            generated_texts.append(simplified)
            is_copy_list.append(is_copy)
            was_regenerated_list.append(regen_level > 0)
            regeneration_level_list.append(regen_level)
            similarity_scores.append(sim)

    # === GUARDAR RESULTADOS ===
    df["generated_text"] = generated_texts
    df["is_copy"] = is_copy_list
    df["was_regenerated"] = was_regenerated_list
    df["regeneration_level"] = regeneration_level_list
    df["similarity_score"] = similarity_scores

    if verbose:
        print(f"\n✅ Generación completada sin errores de memoria")
        print(f"📊 Total textos: {len(df)} | Regenerados: {sum(was_regenerated_list)} | Copias finales: {sum(is_copy_list)}")
        print(f"Promedio de similitud: {sum(similarity_scores)/len(similarity_scores):.3f}")

    return df


In [24]:
df_out = generate_simplifications_v2(
    df=df_test,
    model=fine_tuned_model,
    tokenizer=tokenizer,
    instr_prompt=INSTR_PROMPT,
    batch_size=4, ###subir!! de 2
)


🚀 Generación iniciada (modo seguro, con detección de vacíos)...
✅ Columna 'source_for_generation' creada correctamente


Generando simplificaciones:   0%|          | 0/3 [00:00<?, ?it/s]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


⚠️ Texto vacío detectado en índice 3. Regenerando...


Generando simplificaciones:  33%|███▎      | 1/3 [00:49<01:38, 49.33s/it]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


⚠️ Texto vacío detectado en índice 7. Regenerando...


Generando simplificaciones:  67%|██████▋   | 2/3 [01:40<00:50, 50.14s/it]A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


⚠️ Texto vacío detectado en índice 8. Regenerando...


Generando simplificaciones: 100%|██████████| 3/3 [02:23<00:00, 47.84s/it]


✅ Generación completada sin errores de memoria
📊 Total textos: 10 | Regenerados: 0 | Copias finales: 0
Promedio de similitud: 0.122


In [25]:
df_out

,source,response,source_for_generation,generated_text,is_copy,was_regenerated,regeneration_level,similarity_score
0,Background\nIt is unclear whether blood pressu...,Drug interventions for deliberately altering b...,Background It is unclear whether blood pressur...,defining the problem It is unclear whether blo...,False,False,0,0.382856
1,Background\nIn acute asthma inhaled beta₂‐agon...,Holding chambers (spacers) versus nebulisers f...,Background\nIn acute asthma inhaled beta₂‐agon...,The use of a spacer device to deliver beta ago...,False,False,0,0.007567
2,Background\nApproximately 70% of women will ex...,Absorbable stitches for repair of episiotomy a...,Background\nApproximately 70% of women will ex...,defining the problem The use of sutures during...,False,False,0,0.010609
3,Background\nExtracranial carotid artery stenos...,Patches of different types for carotid patch a...,Background Extracranial carotid artery stenosi...,"Beekeeper, what’s wrong with your heart? Carot...",False,False,0,0.077058
4,Background\nPatients treated with mechanical v...,Topical antibiotics to help reduce death and r...,Background Patients treated with mechanical ve...,defining the problem Patients who receive mech...,False,False,0,0.149672
5,Background\nWhilst the pharmacological profile...,Do medicines used to treat depression help peo...,Background Whilst the pharmacological profiles...,The purpose of this study was to examine the e...,False,False,0,0.004196
6,"Ten trials involving 11,795 women met the incl...",Alternative versus conventional institutional ...,"Ten trials involving 11,795 women met the incl...",defining the role of the alternative setting i...,False,False,0,0.026891
7,Background\nStroke is the third leading cause ...,Early treatment with blood‐thinning drugs for ...,Background Stroke is the third leading cause o...,Background Early treatment with anticoagulants...,False,False,0,0.021354
8,Background\nStriae gravidarum (stretch marks d...,Topical preparations for preventing stretch ma...,Background Striae gravidarum (stretch marks de...,Background Striae gravidarum (stretch marks de...,False,False,0,0.521179
9,We included 38 studies. Based on three studies...,Do acupuncture and related therapies help smok...,We included 38 studies. Based on three studies...,defining the problem Acupuncture is a treatmen...,False,False,0,0.018332


In [26]:
import textwrap
import random

# =====================================================================
# EJEMPLO DE SALIDA SIMPLIFICADA
# =====================================================================
print("\n🧾 Ejemplo de salida simplificada:\n" + "="*50)

# Seleccionamos un ejemplo aleatorio del DataFrame
idx = random.randint(0, len(df_test) - 1)

print(f"\n🔹 Índice: {idx}")

# Función auxiliar para mostrar texto con ancho fijo y saltos de línea
def mostrar_texto_formateado(texto, ancho=150):
    # Reemplaza saltos de línea escapados por saltos reales, si los hay
    texto = texto.replace("\\n", "\n")
    # Divide el texto en líneas y aplica textwrap a cada una
    lineas = []
    for linea in texto.splitlines():
        # Ajusta cada línea a máximo 'ancho' caracteres
        lineas.extend(textwrap.wrap(linea, width=ancho, replace_whitespace=False) or [""])
    print("\n".join(lineas))

# Texto original (hasta 1000 caracteres)
print("\n--- Texto original (source) ---")
mostrar_texto_formateado(df_out.loc[idx, "source"][:1500], ancho=120)

# Texto simplificado generado
print("\n--- Texto simplificado generado ---")
mostrar_texto_formateado(df_out.loc[idx, "generated_text"], ancho=120)

print("\n" + "="*50)



🧾 Ejemplo de salida simplificada:

🔹 Índice: 0

--- Texto original (source) ---
Background
It is unclear whether blood pressure should be altered actively during the acute phase of stroke. This is an update of a
Cochrane review first published in 1997, and previously updated in 2001 and 2008.
Objectives
To assess the clinical effectiveness of altering blood pressure in people with acute stroke, and the effect of different
vasoactive drugs on blood pressure in acute stroke.
Search methods
We searched the Cochrane Stroke Group Trials Register (last searched in February 2014), the Cochrane Database of
Systematic reviews (CDSR) and the Cochrane Central Register of Controlled Trials (CENTRAL) (The Cochrane Library 2014,
Issue 2), MEDLINE (Ovid) (1966 to May 2014), EMBASE (Ovid) (1974 to May 2014), Science Citation Index (ISI, Web of
Science, 1981 to May 2014) and the Stroke Trials Registry (searched May 2014).
Selection criteria
Randomised controlled trials of interventions that aimed to a

In [27]:
df_out

,source,response,source_for_generation,generated_text,is_copy,was_regenerated,regeneration_level,similarity_score
0,Background\nIt is unclear whether blood pressu...,Drug interventions for deliberately altering b...,Background It is unclear whether blood pressur...,defining the problem It is unclear whether blo...,False,False,0,0.382856
1,Background\nIn acute asthma inhaled beta₂‐agon...,Holding chambers (spacers) versus nebulisers f...,Background\nIn acute asthma inhaled beta₂‐agon...,The use of a spacer device to deliver beta ago...,False,False,0,0.007567
2,Background\nApproximately 70% of women will ex...,Absorbable stitches for repair of episiotomy a...,Background\nApproximately 70% of women will ex...,defining the problem The use of sutures during...,False,False,0,0.010609
3,Background\nExtracranial carotid artery stenos...,Patches of different types for carotid patch a...,Background Extracranial carotid artery stenosi...,"Beekeeper, what’s wrong with your heart? Carot...",False,False,0,0.077058
4,Background\nPatients treated with mechanical v...,Topical antibiotics to help reduce death and r...,Background Patients treated with mechanical ve...,defining the problem Patients who receive mech...,False,False,0,0.149672
5,Background\nWhilst the pharmacological profile...,Do medicines used to treat depression help peo...,Background Whilst the pharmacological profiles...,The purpose of this study was to examine the e...,False,False,0,0.004196
6,"Ten trials involving 11,795 women met the incl...",Alternative versus conventional institutional ...,"Ten trials involving 11,795 women met the incl...",defining the role of the alternative setting i...,False,False,0,0.026891
7,Background\nStroke is the third leading cause ...,Early treatment with blood‐thinning drugs for ...,Background Stroke is the third leading cause o...,Background Early treatment with anticoagulants...,False,False,0,0.021354
8,Background\nStriae gravidarum (stretch marks d...,Topical preparations for preventing stretch ma...,Background Striae gravidarum (stretch marks de...,Background Striae gravidarum (stretch marks de...,False,False,0,0.521179
9,We included 38 studies. Based on three studies...,Do acupuncture and related therapies help smok...,We included 38 studies. Based on three studies...,defining the problem Acupuncture is a treatmen...,False,False,0,0.018332


Limpiar el texto generado

In [28]:
def extract_response(gen):
    """
    Extrae la parte del texto posterior a 'Simplified text:' (en cualquiera de sus variantes)
    y limpia encabezados residuales tipo '## Background' o 'Output:'.
    """
    # Captura flexible: acepta ###, espacios, o saltos de línea
    match = re.search(r"#+\s*Output PLS\s*[:\-–]?\s*(.*)", gen, flags=re.IGNORECASE | re.DOTALL)
    if match:
        response = match.group(1).strip()
    else:
        response = gen.strip()

    # Limpieza opcional (cabeceras Markdown o Output:)
    response = re.sub(r"^(Output:|#+\s*(Background|Summary|Question|Answer).*)", "", response, flags=re.IGNORECASE).strip()
    return response

df_out["generated_clean"] = df_out["generated_text"].apply(extract_response)






# 5.Métricas


## Limpieza de Metricas

In [29]:
# Verificar
print(df_test["source"].head(1).values[0])

Background
It is unclear whether blood pressure should be altered actively during the acute phase of stroke. This is an update of a Cochrane review first published in 1997, and previously updated in 2001 and 2008. 
Objectives
To assess the clinical effectiveness of altering blood pressure in people with acute stroke, and the effect of different vasoactive drugs on blood pressure in acute stroke. 
Search methods
We searched the Cochrane Stroke Group Trials Register (last searched in February 2014), the Cochrane Database of Systematic reviews (CDSR) and the Cochrane Central Register of Controlled Trials (CENTRAL) (The Cochrane Library 2014, Issue 2), MEDLINE (Ovid) (1966 to May 2014), EMBASE (Ovid) (1974 to May 2014), Science Citation Index (ISI, Web of Science, 1981 to May 2014) and the Stroke Trials Registry (searched May 2014). 
Selection criteria
Randomised controlled trials of interventions that aimed to alter blood pressure compared with control in participants within one week of a

In [30]:
# ============================================================================
# LIMPIAR DATAFRAME ANTES DE CALCULAR MÉTRICAS
# ============================================================================

print("🧹 LIMPIANDO DataFrame antes de calcular métricas...")
print("="*80)

# Ver estado actual
print(f"Columnas actuales: {len(df_out.columns)}")

# Identificar columnas de métricas a eliminar
metrics_to_remove = [col for col in df_test.columns if any(keyword in col for keyword in [
    'BERT_', 'AlignScore', 'TextStat_', 'FRE', 'FKGL', 'SMOG',
    'CLI', 'DCRS', 'GFI', 'similarity_score'
])]

print(f"Columnas de métricas a eliminar: {len(metrics_to_remove)}")
for col in metrics_to_remove:
    print(f"   - {col}")

# Eliminar columnas de métricas
df_test = df_test.drop(columns=metrics_to_remove, errors='ignore')

print(f"\n✅ DataFrame limpio: {len(df_test.columns)} columnas")
print("\n📋 Columnas restantes:")
for col in df_test.columns:
    print(f"   - {col}")

print("="*80)

# ============================================================================
# AHORA SÍ, CALCULAR MÉTRICAS DESDE CERO
# ============================================================================

print("\n🔄 Calculando métricas desde cero...")
print("="*80)

# Aquí va tu código de cálculo de métricas
# Por ejemplo:
# 1. BERTScore
# 2. AlignScore
# 3. TextStat
# etc.

print("\n✅ Listo para calcular métricas frescas sin duplicados")

🧹 LIMPIANDO DataFrame antes de calcular métricas...
Columnas actuales: 9
Columnas de métricas a eliminar: 0

✅ DataFrame limpio: 2 columnas

📋 Columnas restantes:
   - source
   - response

🔄 Calculando métricas desde cero...

✅ Listo para calcular métricas frescas sin duplicados


## Metricas Todas

In [31]:
!pip install evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00


In [32]:
print(df_out.columns)

Index(['source', 'response', 'source_for_generation', 'generated_text',
       'is_copy', 'was_regenerated', 'regeneration_level', 'similarity_score',
       'generated_clean'],
      dtype='object')


In [33]:
# ============================================================================
# 📊 PASO 1: CALCULAR MÉTRICAS (BERTScore + TextStat + AlignScore)
# ============================================================================

from bert_score import score as bertscore
import textstat
import pandas as pd
import torch
import transformers
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

EMB_MODEL = "sentence-transformers/all-mpnet-base-v2"  # cambia por modelo biomédico si quieres
DEVICE = "cuda" if (torch and torch.cuda.is_available()) else "cpu"
EMB_BATCH = 256   # reducir si OOM en T4 (prueba 128/64)

'''# Fix para AdamW
if not hasattr(transformers, "AdamW"):
    from torch.optim import AdamW
    transformers.AdamW = AdamW

from alignscore import AlignScore
'''

print("="*80)
print("📊 CALCULANDO MÉTRICAS")
print("="*80)

# --------------------------------------------------------
# 🔧 CONFIGURACIÓN INICIAL
# --------------------------------------------------------
col_ref = "response" #pls
col_gen = "generated_text"  # ← Cambié a generated_text (ajusta si es otro)
col_non_pls="source"

# Crear columna limpia si no existe
if col_gen not in df_out.columns and "generated_text" in df_test.columns:
    df_out["generated_clean"] = df_out["generated_text"].str.strip()
    col_gen = "generated_clean"

print(f"\n✓ Columna de referencia: {col_ref}")
print(f"✓ Columna generada: {col_gen}")
print(f"✓ Total textos: {len(df_out)}")

# --------------------------------------------------------
# 🧩 BERTScore
# --------------------------------------------------------
# --------------------------------------------------------
# 🧩 BERTScore 1 — modelo base (por defecto)
# --------------------------------------------------------
print("\n📊 Calculando BERTScore (modelo base)...")

refs = df_out[col_ref].astype(str).tolist()  # textos de referencia (PLS)
preds = df_out[col_gen].astype(str).tolist()  # textos generados
non_pls = df_out[col_non_pls].astype(str).tolist()  # abstract original

# Primer BERTScore: modelo por defecto ("roberta-large") o idioma "en"
P1, R1, F1_1 = bertscore(preds, refs, lang="en", verbose=False)

df_out["BERT_Precision"] = P1.numpy()
df_out["BERT_Recall"] = R1.numpy()
df_out["BERT_F1"] = F1_1.numpy()

print(f"✅ BERTScore (base) calculado")
print(f"   - Precision media: {P1.mean():.3f}")
print(f"   - Recall media: {R1.mean():.3f}")
print(f"   - F1 media: {F1_1.mean():.3f}")


# --------------------------------------------------------
# 🧩 BERTScore 2 — modelo alternativo o comparación distinta
# --------------------------------------------------------
print("\n📊 Calculando BERTScore (modelo alternativo: Longformer)...")

from evaluate import load
bertscore_alt = load("bertscore")

results_alt = bertscore_alt.compute(
    predictions=preds,
    references=refs,
    model_type="allenai/longformer-large-4096-finetuned-triviaqa",
    lang="en"
)

# Convertir los resultados en listas de precisión, recall y F1
P2 = results_alt["precision"]
R2 = results_alt["recall"]
F1_2 = results_alt["f1"]

# Guardar en df_out con nombres distintos
df_out["BERT2_Precision_Longformer"] = P2
df_out["BERT2_Recall_Longformer"] = R2
df_out["BERT2_F1_Longformer"] = F1_2

print(f"✅ BERTScore (Longformer) calculado")
print(f"   - Precision media: {sum(P2)/len(P2):.3f}")
print(f"   - Recall media: {sum(R2)/len(R2):.3f}")
print(f"   - F1 media: {sum(F1_2)/len(F1_2):.3f}")


# --------------------------------------------------------
# 📚 TextStat (legibilidad y complejidad)
# --------------------------------------------------------
print("\n📚 Calculando métricas TextStat...")

def get_textstat_metrics(text):
    try:
        return {
            "flesch_reading_ease": textstat.flesch_reading_ease(text),
            "flesch_kincaid_grade": textstat.flesch_kincaid_grade(text),
            "smog_index": textstat.smog_index(text),
            "coleman_liau_index": textstat.coleman_liau_index(text),
            "automated_readability_index": textstat.automated_readability_index(text),
            "dale_chall_readability_score": textstat.dale_chall_readability_score(text),
            "difficult_words": textstat.difficult_words(text),
            "linsear_write_formula": textstat.linsear_write_formula(text),
            "gunning_fog": textstat.gunning_fog(text),
        }
    except:
        return {key: None for key in [
            "flesch_reading_ease", "flesch_kincaid_grade", "smog_index",
            "coleman_liau_index", "automated_readability_index",
            "dale_chall_readability_score", "difficult_words",
            "linsear_write_formula", "gunning_fog"
        ]}

textstat_metrics = df_out[col_gen].apply(get_textstat_metrics)
df_textstat = pd.DataFrame(textstat_metrics.tolist()).add_prefix("TextStat_")

df_out = pd.concat([df_out, df_textstat], axis=1)

print(f"✅ TextStat calculado ({len(df_textstat.columns)} métricas)")

# --------------------------------------------------------
# 🔗 AlignScore o Equivalente Rapido
# --------------------------------------------------------
# -------- SI NO HAY SCORE, CALCULARLO (GPU / batching) --------
print("\n🔗 Calculando AlignScore (versión ligera)...")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"   Usando: {device}")

def compute_pairwise_align(ref_texts, gen_texts, model_name=EMB_MODEL, batch_size=EMB_BATCH, device=device):
    if SentenceTransformer is None:
        raise ImportError("sentence-transformers no está instalado. Instálalo con: pip install sentence-transformers")
    if len(ref_texts) != len(gen_texts):
        raise ValueError("refs y gens deben tener la misma longitud")

    model = SentenceTransformer(model_name, device=device)
    model.max_seq_length = 512

    def encode_list(texts):
        embs = []
        for i in tqdm(range(0, len(texts), batch_size), desc="Encoding", leave=False):
            batch = texts[i:i+batch_size]
            b_emb = model.encode(batch, convert_to_tensor=True, show_progress_bar=False)
            b_emb = torch.nn.functional.normalize(b_emb, p=2, dim=1)
            embs.append(b_emb)
        return torch.cat(embs, dim=0)

    refs_emb = encode_list(ref_texts)
    gens_emb = encode_list(gen_texts)
    refs_emb = refs_emb.to(device)
    gens_emb = gens_emb.to(device)

    with torch.no_grad():
        cosines = torch.sum(refs_emb * gens_emb, dim=1).clamp(-1.0, 1.0).cpu().numpy()
    return cosines

# Calcular y guardar
align_scores = compute_pairwise_align(non_pls, preds)
df_out["AlignScore"] = align_scores

print(f"✅ AlignScore calculado (cosine embedding)")
print(f"   - Media: {sum(align_scores)/len(align_scores):.3f}")

print("\n" + "="*80)
print("✅ TODAS LAS MÉTRICAS CALCULADAS")
print("="*80)

# ============================================================================
# 📊 PASO 2: RESUMEN DE MÉTRICAS - TABLA PROFESIONAL
# ============================================================================

from tabulate import tabulate

print("\n🔧 Preparando tablas de resumen...")

# Renombrar TextStat con nombres cortos
textstat_rename = {
    "TextStat_flesch_reading_ease": "FRE",
    "TextStat_flesch_kincaid_grade": "FKGL",
    "TextStat_smog_index": "SMOG",
    "TextStat_coleman_liau_index": "CLI",
    "TextStat_dale_chall_readability_score": "DCRS",
    "TextStat_gunning_fog": "GFI",
}

# Aplicar renombrado
rename_dict = {k: v for k, v in textstat_rename.items() if k in df_out.columns}
df_out_display = df_out.rename(columns=rename_dict)

# Seleccionar columnas de métricas
metric_columns = [
    "BERT_Precision",
    "BERT_Recall",
    "BERT_F1",
    "BERT2_F1_Longformer",
    "AlignScore"
] + [v for v in textstat_rename.values() if v in df_out_display.columns]

# Añadir columnas TextStat no renombradas
other_textstat = [col for col in df_out.columns if col.startswith("TextStat_") and col not in textstat_rename]
metric_columns.extend(other_textstat)

# Crear DataFrame de métricas sin duplicados
df_metrics = df_out_display[metric_columns].copy()
df_metrics = df_metrics.loc[:, ~df_metrics.columns.duplicated()]

print(f"✓ Métricas seleccionadas: {len(df_metrics.columns)} columnas")

# ============================================================================
# ✅ CREAR stats_df
# ============================================================================

stats_df = pd.DataFrame({
    'Mean': df_metrics.mean(numeric_only=True),
    'Std': df_metrics.std(numeric_only=True),
    'Min': df_metrics.min(numeric_only=True),
    'Max': df_metrics.max(numeric_only=True),
    'Median': df_metrics.median(numeric_only=True),
}).T

print(f"✓ stats_df creado: {stats_df.shape}")

# ============================================================================
# 📈 TABLA DE MÉTRICAS POR TEXTO (primeros 10)
# ============================================================================

print("\n" + "="*120)
print("📊 MÉTRICAS POR TEXTO (Primeros 10 ejemplos)")
print("="*120)

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)
pd.set_option('display.float_format', '{:.3f}'.format)

print("\n")
print(df_metrics.head(10).to_string())
print("\n" + "="*120)

# ============================================================================
# 📊 ESTADÍSTICAS GLOBALES
# ============================================================================

print("\n" + "="*120)
print("📈 ESTADÍSTICAS GLOBALES")
print("="*120 + "\n")

# Preparar datos para tabulate
table_data = []
for stat_name in ['Mean', 'Std', 'Min', 'Max', 'Median']:
    row = [stat_name] + [f"{val:.3f}" for val in stats_df.loc[stat_name]]
    table_data.append(row)

headers = ['Statistic'] + list(stats_df.columns)

print(tabulate(table_data, headers=headers, tablefmt='grid',
               stralign='center', numalign='center'))
print()

# ============================================================================
# 📊 REFERENCIA COMPARATIVA (Modelos del paper CL4Health 2025)
# ============================================================================

reference_data = [
    ["DeepSeek-R1-7B",        16.99, 22.75, 17.69, 12.31, 14.80, 9.45, 0.7955, 0.8496],
    ["Gemini-1.3-Flash",      9.60,  66.87, 8.75,  9.08,  6.90,  5.94, 0.6333, 0.8474],
    ["GPT-4-32K",            12.48,  48.52, 13.39, 11.20, 10.80, 7.41, 0.7801, 0.8519],
    ["GPT-4o",               11.49,  57.13, 11.16,  9.91,  9.09, 6.88, 0.7383, 0.8527],
    ["GPT-3.5-Turbo-16K",    15.52,  28.08, 17.33, 12.59, 13.91, 8.60, 0.8781, 0.8585],
    ["Llama-3.2-3B",         16.42,  21.96, 18.58, 10.79, 15.73, 9.39, 0.8785, 0.8490],
]

ref_headers = ["Model", "CLI ↓", "FRE ↑", "GFI ↓", "SMOG ↓", "FKGL ↓", "DCRS ↓", "AlignScore ↑", "BERTScore ↑"]

print("\n" + "="*120)
print("📊 TABLA DE REFERENCIA (Valores del paper CL4Health-2025)")
print("="*120 + "\n")

print(tabulate(reference_data, headers=ref_headers, tablefmt="grid",
               stralign="center", numalign="center"))
print()

# ============================================================================
# 🎯 RESUMEN EJECUTIVO
# ============================================================================

print("\n" + "="*100)
print("🎯 RESUMEN EJECUTIVO")
print("="*100)

key_metrics = {
    "Semantic Similarity": {
        "BERT F1": df_metrics["BERT_F1"].mean() if "BERT_F1" in df_metrics.columns else None,
        "AlignScore": df_metrics["AlignScore"].mean() if "AlignScore" in df_metrics.columns else None,
    },
    "Readability": {
        "Flesch Ease (FRE)": df_metrics["FRE"].mean() if "FRE" in df_metrics.columns else None,
        "FK Grade Level": df_metrics["FKGL"].mean() if "FKGL" in df_metrics.columns else None,
        "SMOG Index": df_metrics["SMOG"].mean() if "SMOG" in df_metrics.columns else None,
    },
    "Complexity": {
        "Coleman-Liau": df_metrics["CLI"].mean() if "CLI" in df_metrics.columns else None,
        "Dale-Chall": df_metrics["DCRS"].mean() if "DCRS" in df_metrics.columns else None,
        "Gunning Fog": df_metrics["GFI"].mean() if "GFI" in df_metrics.columns else None,
    }
}

for category, metrics in key_metrics.items():
    print(f"\n📌 {category}:")
    for metric_name, value in metrics.items():
        if value is not None:
            # Emoji según valor
            if "F1" in metric_name or "AlignScore" in metric_name:
                emoji = "✅" if value >= 0.7 else "⚠️" if value >= 0.5 else "❌"
            elif "Flesch" in metric_name or "FRE" in metric_name:
                emoji = "✅" if value >= 60 else "⚠️" if value >= 50 else "❌"
            elif any(x in metric_name for x in ["Grade", "SMOG", "Coleman", "Dale", "Gunning"]):
                emoji = "✅" if value <= 10 else "⚠️" if value <= 12 else "❌"
            else:
                emoji = "📊"

            print(f"   {emoji} {metric_name:.<35} {value:>8.3f}")

print("="*100)

# =====================================================================
# 🎯 RESUMEN EJECUTIVO (con comparación a Llama-3.2-3B)
# =====================================================================

print("\n" + "="*100)
print("🎯 RESUMEN EJECUTIVO (comparado con Llama-3.2-3B)")
print("="*100)

# === 1️⃣ Valores de referencia (Llama-3.2-3B) ===
llama3_ref = {
    "CLI": 16.42,
    "FRE": 21.96,
    "GFI": 18.58,
    "SMOG": 10.79,
    "FKGL": 15.73,
    "DCRS": 9.39,
    "AlignScore": 0.8785,
    "BERT_F1": 0.849
}

# === 2️⃣ Tu resumen de métricas ===
key_metrics = {
    "Semantic Similarity": {
        "BERT_F1": df_metrics["BERT_F1"].mean() if "BERT_F1" in df_metrics.columns else None,
        "AlignScore": df_metrics["AlignScore"].mean() if "AlignScore" in df_metrics.columns else None,
    },
    "Readability": {
        "FRE": df_metrics["FRE"].mean() if "FRE" in df_metrics.columns else None,
        "FKGL": df_metrics["FKGL"].mean() if "FKGL" in df_metrics.columns else None,
        "SMOG": df_metrics["SMOG"].mean() if "SMOG" in df_metrics.columns else None,
    },
    "Complexity": {
        "CLI": df_metrics["CLI"].mean() if "CLI" in df_metrics.columns else None,
        "DCRS": df_metrics["DCRS"].mean() if "DCRS" in df_metrics.columns else None,
        "GFI": df_metrics["GFI"].mean() if "GFI" in df_metrics.columns else None,
    }
}

# === 3️⃣ Mostrar métricas comparadas ===
for category, metrics in key_metrics.items():
    print(f"\n📌 {category}:")
    for metric_name, value in metrics.items():
        if value is not None:
            # Valor de referencia (si existe)
            ref_val = llama3_ref.get(metric_name, None)

            # Emoji según desempeño relativo
            if "F1" in metric_name or "AlignScore" in metric_name:
                emoji = "✅" if value >= 0.7 else "⚠️" if value >= 0.5 else "❌"
            elif "FRE" in metric_name:
                emoji = "✅" if value >= 60 else "⚠️" if value >= 50 else "❌"
            elif any(x in metric_name for x in ["Grade", "SMOG", "CLI", "DCRS", "GFI", "FKGL"]):
                emoji = "✅" if value <= 10 else "⚠️" if value <= 12 else "❌"
            else:
                emoji = "📊"

            # Mostrar línea con comparación
            if ref_val is not None:
                diff = value - ref_val
                arrow = "⬆️" if diff > 0 else "⬇️" if diff < 0 else "➡️"
                print(f"   {emoji} {metric_name:<20} {value:>8.3f}  ({arrow} vs Llama3 {ref_val:>8.3f})")
            else:
                print(f"   {emoji} {metric_name:<20} {value:>8.3f}")

print("="*100)
# ============================================================================
# 💾 EXPORTAR A CSV
# ============================================================================

df_metrics.to_csv("metrics_complete.csv", index=False)
stats_df.T.to_csv("metrics_summary.csv")

print("\n💾 Tablas guardadas:")
print("   - metrics_complete.csv (todas las métricas por texto)")
print("   - metrics_summary.csv (estadísticas resumidas)")
print()

📊 CALCULANDO MÉTRICAS

✓ Columna de referencia: response
✓ Columna generada: generated_text
✓ Total textos: 10

📊 Calculando BERTScore (modelo base)...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ AlignScore calculado (cosine embedding)
   - Media: 0.884

✅ TODAS LAS MÉTRICAS CALCULADAS

🔧 Preparando tablas de resumen...
✓ Métricas seleccionadas: 14 columnas
✓ stats_df creado: (5, 14)

📊 MÉTRICAS POR TEXTO (Primeros 10 ejemplos)


   BERT_Precision  BERT_Recall  BERT_F1  BERT2_F1_Longformer  AlignScore    FRE   FKGL   SMOG    CLI   DCRS    GFI  TextStat_automated_readability_index  TextStat_difficult_words  TextStat_linsear_write_formula
0           0.816        0.841    0.828                0.784       0.957 36.907 15.192 15.021 14.196 14.099 16.780                                18.959                        56                          16.500
1           0.821        0.806    0.813                0.776       0.728 60.342  7.921 11.538  8.652  9.845 11.234                                 7.089                        35                           7.875
2           0.855        0.827    0.841                0.799       0.828 57.297  9.109 11.662 11.603  9.745 10.961             

In [34]:

# Filtrar los casos con AlignScore < 0.3
low_align = df_out[df_out["AlignScore"] < 0.3]

# Mostrar cuántos hay
print(f"🧮 {len(low_align)} ejemplos con AlignScore < 0.3\n")

# Iterar sobre los primeros 10 ejemplos
for i, row in enumerate(low_align[["source", "response", "generated_text", "AlignScore"]].head(10).itertuples(), start=1):
    print(f"\n🔹 Ejemplo {i} | AlignScore: {row.AlignScore:.3f}")
    print("=" * 150)

    # Texto original (source)
    print("\n--- Texto original (source) ---")
    print(textwrap.fill(row.source.replace("\\n", "\n"), width=150, replace_whitespace=False))

    # Respuesta de referencia (ground truth)
    print("\n--- Resumen de referencia (response) ---")
    print(textwrap.fill(row.response.replace("\\n", "\n"), width=150, replace_whitespace=False))

    # Texto generado por el modelo
    print("\n--- Texto simplificado generado ---")
    print(textwrap.fill(row.generated_text.replace("\\n", "\n"), width=150, replace_whitespace=False))

    print("-" * 150)


🧮 0 ejemplos con AlignScore < 0.3



In [35]:
# Filtrar filas donde generated_text está vacío, nulo o solo tiene espacios
empty_generated = df_out[df_out["generated_text"].isna() | (df_out["generated_text"].str.strip() == "")]

# Mostrar cuántos hay
print(f"🧮 {len(empty_generated)} ejemplos con 'generated_text' vacío\n")

# Mostrar las primeras filas con las columnas relevantes
print(empty_generated[["source", "response", "generated_text"]].head(10).to_string(index=False))


🧮 0 ejemplos con 'generated_text' vacío

Empty DataFrame
Columns: [source, response, generated_text]
Index: []


In [36]:


# Aseguramos que las columnas existen antes de iterar
cols_to_show = ["generated_text", "generated_clean", "response", "source"]
cols_present = [c for c in cols_to_show if c in df_out.columns]

print(f"Mostrando columnas: {cols_present}")

for i, row in enumerate(df_out[cols_present].head(5).itertuples(), start=1):
    print(f"\n🔹 Ejemplo {i}")
    print("=" * 150)

    # --- generated_text ---
    if hasattr(row, "generated_text"):
        if not row.generated_text or str(row.generated_text).strip() == "" or str(row.generated_text).lower() == "nan":
            print("⚠️ generated_text está vacío o nulo")
            print("Valor crudo:", repr(row.generated_text))
        else:
            print("\n--- generated_text ---")
            print(textwrap.fill(str(row.generated_text).replace("\\n", "\n"), width=150, replace_whitespace=False))

    # --- generated_clean ---
    if hasattr(row, "generated_clean"):
        print("\n--- generated_clean ---")
        print(textwrap.fill(str(row.generated_clean).replace("\\n", "\n"), width=150, replace_whitespace=False))

    # --- response ---
    if hasattr(row, "response"):
        print("\n--- response ---")
        print(textwrap.fill(str(row.response).replace("\\n", "\n"), width=150, replace_whitespace=False))

    # --- source ---
    if hasattr(row, "source"):
        print("\n--- source ---")
        print(textwrap.fill(str(row.source).replace("\\n", "\n"), width=150, replace_whitespace=False))

    print("-" * 150)


Mostrando columnas: ['generated_text', 'generated_clean', 'response', 'source']

🔹 Ejemplo 1

--- generated_text ---
defining the problem It is unclear whether blood pressure should be altered actively during the acute phase of stroke. This is an update of a Cochrane
review first published in 1997, and previously updated in 2001 and 2008. Objectives To assess the clinical effectiveness of altering blood pressure in
people with acute stroke, and the effect of different vasoactive drugs on blood pressure in acute stroke. Search methods We searched the Cochrane
Stroke Group Trials Register (last searched in February 2014), the Cochrane Database of Systematic reviews (CDSR) and the Cochrane Central Register of
Controlled Trials (CENTRAL) (The Cochrane Library 2014, Issue 2), MEDLINE (Ovid) (1966 to May 2014), EMBASE (Ovid) (1974 to May 2014), Science
Citation Index (ISI, Web of Science, 1981 to May 2014) and the Stroke Trials Registry (searched May 2014). Selection criteria Randomised cont